-  16번째 행의 앞 몇개 컬럼의 값을 못읽는다. 
-  화면 떳을 때 스크롤을 좀 내리고 나면 그건 또 읽는다. 환장

In [14]:
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select 
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

# WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL로 이동
url = "https://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_NTS#"
driver.get(url)

# 페이지 요소와 상호작용
select1 = driver.find_element(By.CSS_SELECTOR, 'a.left-icon02')
select1.click()

time.sleep(1)

# 시작 연도 선택 (2021)
select2 = Select(driver.find_element(By.ID, 'ptsRafYearStart'))
select2.select_by_value("2023")

# 종료 연도 선택 (2023)
select3 = Select(driver.find_element(By.ID, 'ptsRafYearEnd'))
select3.select_by_value("2023")

# 지역 선택 (서울특별시)
select4 = Select(driver.find_element(By.ID, 'ptsRafSido'))
select4.select_by_value("11") 

# 세부 지역 선택 (서울 강서구)
select5 = Select(driver.find_element(By.ID, 'ptsRafSigungu'))
select5.select_by_value("11545")

# 사고유형 체크박스 선택
checkbox1 = driver.find_element(By.CSS_SELECTOR, 'input[name="ACDNT_GAE_CODE"][value="02"]')
checkbox1.click()

checkbox2 = driver.find_element(By.CSS_SELECTOR, 'input[name="ACDNT_GAE_CODE"][value="03"]')
checkbox2.click()

checkbox3 = driver.find_element(By.CSS_SELECTOR, 'input[name="ACDNT_GAE_CODE"][value="04"]')
checkbox3.click()

# 검색 버튼 클릭
search_button = driver.find_element(By.CSS_SELECTOR, 'a.btn-search')
search_button.click()

time.sleep(2)  # 검색 결과 로딩 시간 대기

# 데이터가 로드되기 전에 스크롤을 끝까지 내려서 모든 데이터를 로딩
select7 = driver.find_element(By.CSS_SELECTOR, 'a.btn-minibox')
select7.click()

time.sleep(2)  # 페이지 로드 대기 (5초 대기)

# 총 건수 추출
total_count_text = driver.find_element(By.CSS_SELECTOR, 'div.total-count span').text
total_count = int(total_count_text.replace(',', ''))  # 천 단위 구분 쉼표 제거 후 정수로 변환

print(total_count)

# 새 창이나 탭이 열렸는지 확인하고 전환
main_window = driver.current_window_handle
all_windows = driver.window_handles

for window in all_windows:
    if window != main_window:
        driver.switch_to.window(window)  # 새 창으로 전환

# 키보드 하향 키를 이용해서 끝까지 스크롤
actions = webdriver.ActionChains(driver)

# 스크롤 반복문 시작
data_columns = ["사고번호", "사고일시", "요일", "시군구", "사고내용", "사망자수", 
                "중상자수", "경상자수", "부상신고자수", "사고유형", "법규위반", 
                "노면상태", "기상상태", "도로형태", "가해운전자차종", "가해운전자성별", 
                "가해운전자연령", "가해운전자상해정도"]

# 중간에 저장할 파일 초기화
file_path = "accident_data_full.csv"
if os.path.exists(file_path):
    os.remove(file_path)  # 파일이 있으면 삭제

collected_data = []

accident_info_list = driver.find_element(By.ID, 'accidentInfoListView')
# 첫 번째 행 선택 후 클릭
# first_row = accident_info_list.find_elements(By.CSS_SELECTOR, '.slick-row')[0]
# first_row.click()
accident_info_list.click()
while True:
    # 스크롤 후 새로운 데이터 로드가 완료될 때까지 대기
    time.sleep(2)  # 각 스크롤 후 대기
    
    # 스크롤 후 매번 새로운 데이터 추출
    accident_info_list = driver.find_element(By.ID, 'accidentInfoListView')
    rows = accident_info_list.find_elements(By.CSS_SELECTOR, '.slick-row')
    
    # 행 데이터를 수집
    new_data = []
    for idx, row in enumerate(rows):
        try:
            cells = row.find_elements(By.TAG_NAME, 'div')  # 각 행의 셀 데이터를 추출
            
            row_data = [cell.text if cell.text else 'N/A' for cell in cells]  # 빈 셀이 있으면 'N/A'로 대체
            new_data.append(row_data)
            if idx in [14, 15, 16]:
                print(len(cells), end=", ")
                print(cells[0].get_attribute("outerHTML"), end=",")
                print(cells[0].text)

                
                
        except Exception as e:
            print(f"Error at row: {row}, error: {str(e)}")
            continue
    
    # 새롭게 수집된 데이터를 pandas DataFrame으로 변환
    new_df = pd.DataFrame(new_data, columns=data_columns)
    
    # 중복을 제거하고 데이터 추가
    collected_data.append(new_df)
    
    
    df = pd.concat(collected_data).drop_duplicates(subset=['사고번호'])
    # print(len(collected_data), len(df), total_count)
    # print(new_df)

    # 스크롤 내리기
    actions.send_keys(Keys.PAGE_DOWN).perform()
    break
    # 모든 데이터가 수집되었는지 확인 (현재까지 수집된 데이터의 길이로 비교)
    # if len(df) >= 200:
    #     print(f"총 {total_count}건의 데이터를 모두 로드했습니다.")
    #     break

# 중복을 제거한 최종 데이터 저장
final_df = pd.concat(collected_data).drop_duplicates(subset=['사고번호'])

# 데이터 CSV로 저장
final_df.to_csv(file_path, index=False)

# 로딩이 완료된 데이터의 개수 확인
print(f"로드된 데이터 개수: {len(final_df)}")
print(final_df.head(20))  # 상위 5개의 데이터를 확인

# WebDriver 종료
driver.quit()


876
18, <div class="ui-widget-content slick-row even" style="top:350px"><div class="slick-cell l0 r0">2023010500100367</div><div class="slick-cell l1 r1">2023년  1월  5일 18시</div><div class="slick-cell l2 r2">목요일</div><div class="slick-cell l3 r3">서울특별시 금천구 시흥동</div><div class="slick-cell l4 r4">중상사고</div><div class="slick-cell l5 r5">0</div><div class="slick-cell l6 r6">1</div><div class="slick-cell l7 r7">0</div><div class="slick-cell l8 r8">0</div><div class="slick-cell l9 r9">차대차 - 기타</div><div class="slick-cell l10 r10">신호위반</div><div class="slick-cell l11 r11"> 건조</div><div class="slick-cell l12 r12">맑음</div><div class="slick-cell l13 r13">교차로 - 교차로안</div><div class="slick-cell l14 r14">이륜</div><div class="slick-cell l15 r15">남</div><div class="slick-cell l16 r16">53세</div><div class="slick-cell l17 r17">중상</div></div>,Error at row: <selenium.webdriver.remote.webelement.WebElement (session="9c96d5bebcad987fd972562b574baf7a", element="f.4B3E9671138B0ADB007C4E64A717C172.d.EF697B84CA2